In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
df=pd.read_csv("Q2_Fraud_check.csv")
df

In [ ]:
#droping empty rows
df.dropna(subset=['Undergrad'], inplace=True)
df.dropna(subset=['Marital.Status'], inplace=True)
df.dropna(subset=['Taxable.Income'], inplace=True)
df.dropna(subset=['City.Population'], inplace=True)
df.dropna(subset=['Work.Experience'], inplace=True)
df.dropna(subset=['Urban'], inplace=True)
df

In [ ]:
# Categorical Variable Handling (One-Hot Encoding)
df = pd.get_dummies(df, columns=['Undergrad', 'Marital.Status', 'Urban'], drop_first=True)

# Target Variable Transformation
df['Taxable.Income'] = df['Taxable.Income'].apply(lambda x: 0 if x <= 30000 else 1)

# Feature Scaling
scaler = MinMaxScaler()
df[['Work.Experience', 'City.Population']] = scaler.fit_transform(df[['Work.Experience', 'City.Population']])

# Data Splitting
X = df.drop('Taxable.Income', axis=1)
y = df['Taxable.Income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=30)

# Decision Tree Model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)

# Display the classification report for model evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define hyperparameter search space
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),  # Decision Tree Classifier
    param_distributions=param_dist,           # Hyperparameter search space
    n_iter=10,                                # Number of random samples
    cv=5,                                    # Cross-validation folds
    scoring='accuracy',                      # Scoring metric
    random_state=42                           # Random seed for reproducibility
)

# Perform hyperparameter tuning using RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best hyperparameters and the best model
best_params_random = random_search.best_params_
best_model_random = random_search.best_estimator_

# Evaluate the tuned model using RandomizedSearchCV
y_pred_tuned_random = best_model_random.predict(X_test)

# Display the classification report for the tuned model
print("Randomized Tuned Decision Tree - Classification Report:")
print(classification_report(y_test, y_pred_tuned_random))